In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
abbr = 'UT'

In [4]:
file = 'utah2018.csv'

Read in federal level data

In [ ]:
fiscal = pd.read_sas('../../data/fiscal2018', format = 'sas7bdat', encoding='iso-8859-1')

Generate list of districts in the state in the federal data

In [ ]:
state_fiscal = fiscal[(fiscal['STABBR'] == abbr) & (fiscal['GSHI'] == '12')]

In [ ]:
len(state_fiscal)

Read in state level data

In [ ]:
state_grads = pd.read_csv('../../data/state_data_raw/' + file)

In [ ]:
state_grads.head()

Reset columns.

Filter results.

In [ ]:
#state_grads = state_grads[(state_grads['CALC_FOR_STATE_ACCT'] == 'No')]

Select and rename columns.

In [ ]:
state_grads = state_grads[['LEA', 
                           '2018 Cohort Count',
                           '2018 Graduation Rate']]

In [ ]:
state_grads.columns = ['District Name', 'Total', 'Graduation Rate']

In [ ]:
state_grads.info()

Convert data types.

In [ ]:
state_grads['Graduation Rate'] = state_grads['Graduation Rate'].astype(str).str.replace('%', '')
state_grads['Graduation Rate'] = state_grads['Graduation Rate'].astype(str).str.replace('>=', '')
state_grads['Graduation Rate'] = state_grads['Graduation Rate'].astype(str).str.replace('<', '')

state_grads['Graduation Rate'] = state_grads['Graduation Rate'].astype(str).str.replace('40-49', '40')
state_grads['Graduation Rate'] = state_grads['Graduation Rate'].astype(str).str.replace('70-79', '70')
state_grads['Graduation Rate'] = state_grads['Graduation Rate'].astype(str).str.replace('80-89', '80')

In [ ]:
state_grads['Total'] = pd.to_numeric(state_grads['Total'])
state_grads['Graduation Rate'] = pd.to_numeric(state_grads['Graduation Rate']) / 100

Check for matches and non-matches in the two lists. 

In [ ]:
# state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.upper().str.strip()
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.upper().str.strip()

In [ ]:
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' City', '')
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' Schools', '')
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' School District', '')
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' School System', '')
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' Municipal', '')
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' Special', '')

# state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' City', '')
# state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' Schools', '')
# state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' School District', '')
# state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' Municipal', '')

In [ ]:
matches = [name for name in list(state_grads['District Name']) if name in list(state_fiscal['NAME'])]
matches.sort()
len(matches)

In [ ]:
A = [name for name in list(state_grads['District Name']) if name not in list(state_fiscal['NAME'])]
A.sort()
A

In [ ]:
B = [name for name in list(state_fiscal['NAME']) if name not in list(state_grads['District Name'])]
B.sort()
B

Make any additional matches I can find.

In [ ]:
state_fiscal_rename = {
    'American Preparatory Academy - LEA' : 'American Preparatory Academy',
    'Davinci Academy' : 'DaVinci Academy'
}

In [ ]:
state_fiscal = state_fiscal.replace(state_fiscal_rename)

Merge federal and state data, keeping only matches between the two. 

In [ ]:
state_grads_merged = pd.merge(state_fiscal, state_grads, how='inner', left_on='NAME', right_on='District Name')

Save cleaned data. 

In [ ]:
state_grads_merged.to_csv('../../data/state_data_merged/' + abbr + '.csv', index=False)